# Выбор архитектуры!

Итак, передо мной встал вопрос о более грамотной архитектуре моего RL-кода.

Основные концепции:
* модульность - основное требование. Агент должен собираться из модулей как из кубиков лего.
* модификация структуры алгоритма должна выражаться в виде модуля. if double_dqn: do_one_thing() else: do_another_thing() здесь не выживет.
* в частности, у пользователя должна быть возможность в две строчки подменить какой-нибудь метод агента (например, лосс-функцию).
* я осознаю, что в питоне можно сделать всё, но нужно адекватное, чистое и элегантное решение. Разрешается, при необходимости, спрятать требуемую мутотень в условно базовый класс, если этим будет удобно пользоваться в том числе при создании новых модулей.

[внимание, код далее условный и не предназначен для запуска]

In [1]:
env = gym.make("CartPole-v0")

CartpoleNN = nn.Sequential(
                nn.Linear(4, 20),
                nn.ELU(),
                nn.Linear(20, 20),
                nn.ELU())

# Вариант №1. Старый вариант

* Агент есть один класс.
* Модули получаются за счёт динамического наследования друг от друга.
* Полученный агент принимает все гиперпараметры на вход в виде конфиг-словаря

In [1]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
# отсутствуют!
# Возможно добавить немного костылей, чтобы делать проверку, например, что все заданные в конфиги гиперпараметры
# действительно используются и не произошло опечатки в названии (это местная проблема)

In [ ]:
# это пишет пользователь, придумавший свою лосс-функцию
def MyLoss(parclass):                        # соглашение: модуль может быть унаследован от произвольного класса
    class MyLoss(parclass):        
        def loss(self, prediction, truth):
            return self.config["hp"]         # соглашение: гиперпараметры хранятся в self.config
    return MyLoss                            

In [ ]:
# создание агента
config = {
    "env": env,
    "network": CartpoleNN,
    "buffer_size": 10^4,
    "optimizer": Adam,
    "target_update_frequency": 100,
    "hp": 42
}

Agent = Runner()
Agent = Replay(Agent)
Agent = DQN(Agent)                    # неявно создаётся голова нейросетки, оптимизатор, пайплайн обучения нейросети...
Agent = Target(Agent)
Agent = eGreedy(Agent)
Agent = MyLoss(Agent)

agent = Agent(config)
agent.learn(1000000)

Недостатки (причины появления данного файла):

0.1) "неявное" создание модулей.

0.2) гиперпараметры модулей слились в одну кучу. Непонятно, к какому модулю какой гиперпараметр относится.

1) очевидно, сделать в системе два DQN или два оптимизируемых функционала можно только через одно место.

2) всё лежит в одном объекте и рискует перезаписать переменные предыдущих агентов

3) пользователю может быть неочевидно, в каком порядке нужно перечислять модули

# Вариант №1.1. Альтернативный старый вариант

(+) минимальное количество строчек кода
(+) аккуратный синтаксис
(-) основные проблемы 1-3 предыдущего варианта

In [1]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
# всё ещё отсутствуют!

In [ ]:
# это пишет пользователь, придумавший свою лосс-функцию
def MyLoss(parclass, hp=42):                       # соглашение: модуль может быть унаследован от произвольного класса
    class MyLoss(parclass):
        def loss(self, prediction, truth):
            return hp
    return MyLoss

In [ ]:
Agent = Runner(env=env)
Agent = Replay(Agent, buffer_size=10^4)
Agent = Network(Agent, network=CartpoleNN, optimizer=Adam)
Agent = DQN(Agent)
Agent = Target(Agent, target_update_frequency=100)
Agent = eGreedy(Agent)
Agent = MyLoss(Agent, hp=42)

agent = Agent()
agent.learn(1000000)

# Вариант №затыка. Пытаемся разбить на отдельные блоки

Чтобы решить проблему, что все модули стакаются в один объект, сделаем так:
* модули получают в качестве гиперпараметров ссылки на другие необходимые модули (зависимости от других модулей придётся указывать явно)
* от модулей всё ещё можно получать новые модификации путём наследования

Пробуем лобовой подход:
* гиперпараметры передаём при создании класса
* ссылки на другие модули (уже объекты классов) передаём в конструктор модуля

Соответственно, перед передачей ссылки нужно создать модуль.

In [1]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
# всё ещё отсутствуют!

In [ ]:
# это пишет пользователь, придумавший свою лосс-функцию
def MyLoss(parclass, hp=42):
    class MyLoss(parclass):
        def loss(self, prediction, truth):
            return hp
    return MyLoss

In [4]:
# в нашем примере пусть будет 4 модуля: runner, replay, network, dqn
runner = Runner(env=env)()
replay = Replay(buffer_size=10^4)(runner)      # buffer_size - гиперпараметр, ссылка на модуль runner идёт в конструктор
network = Network(optimizer=Adam)()

dqn = DQN()                                    # пока это класс
dqn = Target(dqn, target_update_frequency=100) # улучшаем класс
dqn = MyLoss(dqn, hp=42)                       # ещё улучшаем
dqn = dqn(replay, network)                     # вызываем конструктор, создавая модуль и передавая необходимые ссылки
eGreedy = ?!?                                  # а вот и засада

runner.run(1000000)

В чём здесь проблема: в рекурсивной зависимости. Runner-у должно быть принипиально пофиг, сколько ещё модулей есть в системе. При этом ему нужна стратегия (метод def act(self, s)), которым мы, собственно, играем в игры. От Runner-а зависит реплей буффер, от буффера DQN. Но затем нужно подцепить в Runner ссылку на DQN (а точнее даже как-то на eGreedy)...

eGreedy можно в рамках концепции полагать или наследником DQN, или наследником Runner-а, но проблему это не решает. В первом случае непонятно, как обновить метод runner.act уже после создания runner-а (писать runner.act = dqn.act, очевидно, отвратительнейший вариант, и в более сложных алгоритмах подобные рекурсивные связи - частое явление (например, Twin DQN)). Во втором случае runner уже создан, и как унаследоваться от класса и элегантно "обновить" его экземпляр непонятно.

# Вариант №2. Класс System

Чтобы решить проблему, делаем так:
* все модули наследуются от базового класса RLmodule
* класс System компонует модули в одну рабочую систему
* от модулей всё ещё можно получать новые модификации путём наследования, зависимости от других модулей придётся указывать явно.

Тогда необходимо предоставить интерфейс связывания модулей.

Первый вариант получается немного упоротым.

In [ ]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
class System:
    def __init__(self):
        self.modules = []
    
    def add(self, module):
        # module - класс (!), унаследованный от RLmodule
        
        # добавляет модуль в список модулей
        self.modules += [module]
        
        # возвращает ID
        return len(self.modules)
    
    def update(self, module_id, update, args):
        # берёт модуль с ID=module (это класс), наследует от него update и кладёт по тому же ID.
        
    def create(self):
        # инициализирует (вызывает конструкторы) все модули

class RLmodule:
    def __init__(self, system)
        self.system = system
        
# зачем: способ обращения к другому модулю будет выглядеть тогда как-то так:
# на примере вызова реплей-буффера из DQN:
self.system[self.replay].sample()

In [ ]:
# это пишет пользователь, придумавший свою лосс-функцию
def MyLoss(parclass=RLmodule, hp=42):       # соглашение: модуль должен быть унаследован от RLmodule или производного
    class MyLoss(parclass):
        def loss(self, prediction, truth):
            return hp
    return MyLoss

In [4]:
system = System()
runner = system.add(Runner(env=env))                                # Runner возвращает класс, унаследованный от RLmodule
replay = system.add(Replay(runner=runner, buffer_size=10^4))        # Replay тоже, но ещё он запоминает ID runner-а
network = system.add(Network(optimizer=Adam))                       
dqn = system.add(DQN(replay, network))                              # DQN запоминает ID модулей replay, network в системе
dqn = system.update(dqn, Target, {"target_update_frequency": 100})  # system.update наследует Target от DQN
dqn = system.update(dqn, MyLoss, {"hp": 42})                        # тоже самое
runner = system.update(runner, eGreedy, {"greedy_agent": dqn})      # тоже самое, но eGreedy теперь ещё нужно подсоединитсья к dqn

agent = system.create()
agent.learn(1000000)

(+) проблемы решены

(-) system.add и system.update повсюду, причём нужно думать, что где ставить

(-) разный синтаксис передачи гиперпараметров и подсоединений

(-) непонятно, где подсоединяются модули, а где гиперпараметры

(-) очень мутно и тяжеловесно

# Вариант №3. Процедура сборки

In [ ]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
class System:
    def create(self, modules):
        # см. вызов в примере далее

In [ ]:
# это пишет пользователь, придумавший свою лосс-функцию
def MyLoss(parclass, hp=42):
    class MyLoss(parclass):
        def loss(self, prediction, truth):
            return hp
    return MyLoss

In [4]:
runner = Runner(env=env)
replay = Replay(buffer_size=10^4)
network = Network(optimizer="Adam")
dqn = DQN()
dqn = Target(dqn, target_update_frequency=100)
dqn = MyLoss(dqn, hp=42)
runner = eGreedy(runner)

dqn = dqn()
runner = runner()
replay = replay()
network = network()

agent = System().create(
    (runner, {"dqn": dqn})
    (replay, {"runner": runner}),
    (network, {}),
    (dqn, {"runner": runner, "network": network})
)
agent.learn(1000000)

По сути, System делает сейчас что-то вроде такого: \
runner.dqn = dqn \
replay.runner = runner \
dqn.runner = runner \
dqn.network = network

Это порешало многие проблемы, но процедура инициализации очень громоздкая.

(-) вызов System.create костылющий

(-) сначала блок создания классов, потом блок создания объектов, потом большой вызов System...

# Вариант №4. Сборка по связям

In [ ]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
class System:
    def __init__(self, **kwargs):
        # см. вызов в примере далее

In [ ]:
# пользователю становится тяжелее...
def MyLoss(name, hp=42):
    def MyLoss(parclass):
        class MyLoss(parclass):
            def __init__(self, system):
                super().__init__(self, system, name)   # нужно указывать явно, чтобы передать name...
            
            def loss(self, prediction, truth):
                return hp
        return MyLoss
    return MyLoss

In [4]:
agent = System(
    Runner("runner", env=env),
    eGreedy("runner", dqn="dqn"),
    Replay("replay", runner="runner", buffer_size=10^4),
    Network("network", optimizer=Adam),
    DQN("dqn", runner="runner", replay="replay"),
    Target("dqn", target_update_frequency=100),
    MyLoss("dqn", hp=42)
)

agent.learn(1000000)

Что делает System: для каждого аргумента наследует очередной элемент списка от предыдущего, если их имена совпадают, заменяет поля-токены соответственно именам, поданным в System.

(-) отвратительнейшее оформление нового модуля (функция, возвращающая функцию, возвращающую класс + явный конструктор)

# Вариант №5. Искусственное наследование

Не наследуем модули одни от других. Все модули просто унаследованы напрямую от RLmodule, и искусственный механизм наследования как-то (?) запихнут в System и RLmodule

In [ ]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
class RLmodule:
    def __init__(self, name):
        self._name = name

class System:
    def __init__(self, **kwargs):
        # см. вызов в примере далее

In [ ]:
class MyLoss(RLmodule):
    def __init__(self, name, hp=42):
        super().__init__(self, name)
        self.hp = hp

    def loss(self, prediction, truth):
        return self.hp

In [4]:
agent = System(
    Runner("runner", env=env),
    Replay("replay", runner="runner", buffer_size=10^4),
    Network("network", optimizer=Adam),
    DQN("dqn", runner="runner", replay="replay"),
    Target("dqn", target_update_frequency=100),
    MyLoss("dqn", hp=42),
    eGreedy("runner", dqn="dqn"),
)

agent.learn(1000000)

(+) получилась какая-то приемлемая внешность фреймворка

(-) без наследования сами модули будут иметь кучу костылей. В частности, у них не будет (прямого) доступа к полям "предков", и это надо будет костылить в искусственном наследовании. И к тому, как его делать, тоже много вопросов :(

**ВЫВОДЫ:** один вариант хуже другого.

# Вариант №6: Смесь вариантов

In [73]:
from collections import defaultdict

class RLmodule():
    def __init__(self, name, system):
        self.system = system
        self.name = name
        
    def EMIT(self, message, *args, **kwargs):
        self.system.SEND(self.name, message, *args, **kwargs)
        
    def CATCH(self, name, message, subscriber):
        self.system.CATCH(name, message, subscriber)
        
    def __getitem__(self, module_name):
        return self.system.modules[module_name]
        
class System():
    def __init__(self, **kwargs):
        self.modules = {}
        self.subscribers = defaultdict(list)
        for name, module in kwargs.items():
            self.modules[name] = module(name, self)
    
    def SEND(self, name, message, *args, **kwargs):
        for subscriber in self.subscribers[(name, message)]:
            subscriber(*args, **kwargs)
            
    def CATCH(self, name, message, subscriber):
        self.subscribers[(name, message)].append(subscriber)

In [74]:
# ПОЛНОЦЕННЫЙ ТЕСТ
def Runner(parclass):    
    class Runner(parclass):
        def act(self, s):
            return 42
        
        def run(self):
            self.EMIT("transition", self.act(0))
    return Runner

def Replay(parclass, runner):        
    class Replay(parclass):
        def __init__(self, name, system):
            super().__init__(name, system)
            self.CATCH(runner, "transition", self.see)

        def see(self, a):
            print("Replay catched ", a)
            self.EMIT("batch", a / 2)
    return Replay

def Network(parclass):
    class Network(parclass):
        pass
    return Network

def DQN(parclass, replay, network):
    class DQN(parclass):
        def __init__(self, name, system):
            super().__init__(name, system)
            self.CATCH(replay, "batch", self.batch)
            
        def loss(self, b):
            return b
            
        def batch(self, b):
            print("batch: ", self.loss(b))
    return DQN

def Target(parclass):
    class Target(parclass):
        def loss(self, b):
            return b / 2
    return Target

def eGreedy(parclass, dqn):
    class eGreedy(parclass):
        def act(self, s):
            return self[dqn].loss(s)
    return eGreedy

In [75]:
runner = Runner(RLmodule)
replay = Replay(RLmodule, runner="runner")
network = Network(RLmodule)
dqn = DQN(RLmodule, replay="replay", network="network")
dqn = Target(dqn)
runner = eGreedy(runner, dqn="dqn")

system = System(
    runner = runner,
    replay = replay,
    network = network,
    dqn = dqn
)

In [76]:
system.modules['runner'].run()

Replay catched  0.0
batch:  0.0


**Выводы:** ну не знаю, но этот вариант пока, кажется, лучший.

# Ещё попытка

In [68]:
class RLmodule():
    def __init__(self, name, system):
        self.system = system
        self.system.modules[name] = self
        self.name = name
        
    def __getitem__(self, module_name):
        return self.system.modules[module_name]
        
class System():
    def __init__(self, **kwargs):
        self.modules = {}

In [69]:
# ПОЛНОЦЕННЫЙ ТЕСТ
class Runner(RLmodule):
    def __init__(self, name, system, policy=None, listeners=[]):
        super().__init__(name, system)
        self.policy = name if policy is None else policy
        self.listeners = listeners
        
    #def add_listener(self, module):
    #    self.listeners.append(module)
    #    return module
    
    def act(self, s):
        return 0

    def run(self):
        s = 42
        a = self[self.policy].act(s)
        for listener in self.listeners:
            self[listener].see(a)
    
class Replay(RLmodule):
    def __init__(self, name, system):
        super().__init__(name, system)

    def see(self, a):
        self.a = a
        print("Replay catched ", a)
        
    def sample(self):
        return self.a + 1
    
class BatchSampler(RLmodule):
    def __init__(self, name, system, replay, listeners=[]):
        super().__init__(name, system)
        self.replay = replay
        self.listeners = listeners
        
#     def add_listener(self, module):
#         self.listeners.append(module)
#         return module
        
    def see(self, a):
        batch = self[self.replay].sample()
        print("Generated batch: ", batch)
        for listener in self.listeners:
            self[listener].process_batch(batch)

class Network(RLmodule): 
    def __init__(self, name, system):
        super().__init__(name, system)
        self.heads = []
        
    def add_head(self, module):
        self.heads.append(module)
        return module
        
    def process_batch(self, batch):
        loss = 0
        for head in self.heads:
            loss += head.loss(batch)
        print("Loss: ", loss)

class DQN(RLmodule):
    def __init__(self, name, system, evaluator=None):
        super().__init__(name, system)
        self.evaluator = name if evaluator is None else evaluator
    
    def act(self, s):
        return s
    
    def evaluate(self, batch):
        return batch
    
    def loss(self, batch):
        return self[self.evaluator].evaluate(batch)

class Target(RLmodule):
    def __init__(self, name, system, frozen_network):
        super().__init__(name, system)
        self.frozen_network = frozen_network
    
    def see(self, a):
        print("target network updated")
    
    def evaluate(self, b):
        return b / 2

class eGreedy(RLmodule):
    def __init__(self, name, system, greedy_policy):
        super().__init__(name, system)
        self.greedy_policy = greedy_policy
    
    def act(self, s):
        return self[self.greedy_policy].act(s) * 10

In [70]:
system  = System()
runner  = Runner("runner", system, policy="eGreedy", listeners=["replay", "sampler", "target"])
replay  = Replay("replay", system)
sampler = BatchSampler("sampler", system, replay="replay", listeners=["network"])
network = Network("network", system, heads=["q_head"], losses=["dqn"])
q_head  = QHead("q_head", system)
dqn     = DQN("dqn", system, evaluator="target")
target  = Target("target", system, frozen_network="network", head="q_head")
policy  = eGreedy("eGreedy", system, greedy_policy="dqn")
runner.run()

Replay catched  420
Generated batch:  421
Loss:  210.5
target network updated


Кажись, что-то наклёвывается.

In [71]:
system.modules

{'runner': <__main__.Runner at 0x1bf288e00f0>,
 'replay': <__main__.Replay at 0x1bf288e0358>,
 'sampler': <__main__.BatchSampler at 0x1bf288e0390>,
 'network': <__main__.Network at 0x1bf26d773c8>,
 'dqn': <__main__.DQN at 0x1bf26d77748>,
 'target': <__main__.Target at 0x1bf2896d390>,
 'eGreedy': <__main__.eGreedy at 0x1bf2896d9b0>}

# СНОВА ВАРИАНТ

In [1]:
# СКРЫТЫЕ ВНУТРЕННОСТИ АРХИТЕКТУРЫ:
# всё ещё отсутствуют!

In [113]:
def Runner():
    class Runner():
        def __init__(self):
            self.storage = {}
        
        def act(self, s):
            return 0
        
        def see(self, a):
            pass
        
        def learn(self, a):
            a = self.act(42)
            self.see(a)
            
        def optimize(self, nname, a):
            pass
    return Runner

def Replay(parclass):
    class Replay(parclass):
        def process_batch(self, a):
            pass
        
        def see(self, a):
            super().see(a)
            self.process_batch(a)
    return Replay

def Network(parclass, name):
    class Network(parclass):
        def __init__(self):
            super().__init__()
            assert name not in self.storage, name
            self.storage[name] = 0
            print("added: ", name)
        
        def optimize(self, nname, a):
            super().optimize(nname, a)
            if nname == name:
                print(nname, " loss is ", self.storage[name])
        
        def process_batch(self, a):
            super().process_batch(a)
            print("Launching optimization with ", name)
            self.optimize(name, a)
    return Network

def DQN(parclass, use_network):
    class DQN(parclass):
        def loss(self, a):
            return a - 1
        
        def optimize(self, nname, a):
            if nname == use_network:
                self.storage[nname] += self.loss(a)
            super().optimize(nname, a)
    return DQN

def Target(parclass):
    class Target(parclass):
        def loss(self, a):
            return a - 10
    return Target

def eGreedy(parclass):
    class eGreedy(parclass):
        def act(self, s):
            return 42
    return eGreedy

In [114]:
Agent = Runner()
Agent = Replay(Agent)
Agent = Network(Agent, "hi!")
Agent = DQN(Agent, "hi!")
Agent = Target(Agent)
Agent = eGreedy(Agent)
Agent = Network(Agent, "weell")

agent = Agent()
agent.learn(1000000)

added:  hi!
added:  weell
Launching optimization with  hi!
hi!  loss is  32
Launching optimization with  weell
weell  loss is  0


In [115]:
agent.storage

{'hi!': 32, 'weell': 0}